In [ ]:
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
eps, weight_tg, weight_x3, weight_x5 = 10**(-3), 0.0, 0.0, 0.0 # last couple =0.0
weight = 1.0 - weight_tg - weight_x3 - weight_x5

def transform(x):
    x['cp_type'] = x[x['cp_type']=='trt_cp']
    x.pop('cp_type')
    x['cp_dose'] = x['cp_dose'].replace({'D1':-1, 'D2':1})
    x['cp_time'] = x['cp_time'] // 24
    x.join(pd.get_dummies(x['cp_time']))
    x.pop('cp_time')
    return x
def cube(x):
    x /= 10.0
    return 10.0 * x*x*x

def tgfunc(x):
    return  np.tan(np.pi *x/ 4.0) 
def application_transform(x):
    x = 2*x-1.0
    x1 = weight_tg*tgfunc(x) + (weight_x3 + weight_x5*x**2)*x**3 + weight*x
    x1 = x1/2 + 0.5
    if x1 < eps:
        x1 = eps
    elif x1 >= 1 - eps:
        x1 = 1 - eps
    return x1

X_all = pd.read_csv('/kaggle/input/lish-moa/train_features.csv')
y_all = pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv')
whatisit = pd.read_csv('/kaggle/input/lish-moa/train_targets_nonscored.csv')
X = transform(X_all[X_all.columns[1:]])
Y = y_all[y_all.columns[1:]]


for random_state in range(2020,2070,10):
    X_train, submission, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, 
                                                        random_state=random_state)
    pseudoinverse_X = np.linalg.pinv(X_train)
    subm = np.dot(submission,np.dot(pseudoinverse_X,Y_train))

    dim = 206*len(submission) 
    df = pd.DataFrame(subm)
    df = df.applymap(application_transform)
    df = np.array(df).reshape(dim)
    
    print(log_loss(np.array(Y_test).reshape(dim),df))

print([eps, weight, weight_tg, weight_x3, weight_x5, len(X.columns)])



In [ ]:
# argmax = []
# B = np.dot(pseudoinverse_X,Y_train)
# for i in range(B.shape[0]):
#     for j in range(B.shape[1]):
#         if B[i,j] > 0.005:
#             argmax.append([i,j])
# len(argmax)            


### Здесь происходит добавление новых признаков x_i^3
# X3 = X.applymap(cube)
# X3.columns = ['cube'+ch for ch in X3.columns]
# X = X.join(X3)

## Здесь происходит добавление новых признаков ((1.0 + x_i/10.0)*(1.0 + x_j/10.0) - 1.0)*10.0
# for t in argmax:
#     i, j = t[0], t[1]
#     X[str(i)+'-'+str(j)] = ((1.0+X[X.columns[i]]/10.0)*(1.0+X[X.columns[j]]/10.0)-2.0)*5.0 

## Здесь происходит добавление новых признаков x_i + x_j
# for t in argmax:
#     i, j = t[0], t[1]
#     X[str(i)+'-'+str(j)] = (X[X.columns[i]]*X[X.columns[j]])*0.1 

In [ ]:
np.linalg.svd(np.dot(pseudoinverse_X,Y_train))

In [ ]:
submission.to_csv('submission.csv', index=False) #, float_format="%.10f")